### קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [4]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [5]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [6]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [7]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [9]:

def split_index_by_taz(index,taz,min_prec,col_name_to_split):
    index['index_area']=index.area
    
    taz['taz_area']=taz.area

    index_taz=index.overlay(taz[['Taz_num','taz_area','geometry']])

    index_taz['small_area']=index_taz.area

    index_taz['precent_from_big_index']=index_taz['small_area']/index_taz['index_area']
    
    index_taz['precent_from_big_taz']=index_taz['small_area']/index_taz['taz_area']

    index_taz=index_taz.loc[(index_taz['precent_from_big_index']>min_prec)|(index_taz['precent_from_big_taz']>0.9)]
    
    index_taz=index_taz[['id','Taz_num','precent_from_big_index']]

    new_big=index_taz.groupby(['id']).sum()

    index=index.set_index('id')
    index['new_big']=new_big['precent_from_big_index']

    index=pd.merge(index.reset_index(),index_taz,on='id')

    for c in col_name_to_split:
        index['{}'.format(c)]=index['{}'.format(c)]*(index['precent_from_big_index']/index['new_big'])
        
        
    return index

In [10]:
def sum_scenarios(sum_df,df,scenario_name,col_name,col_need,num_round):

    df=round(pd.pivot_table(df,index=col_name,aggfunc=sum)[col_need],num_round)

    df['scenario']=scenario_name
    
    sum_df=pd.concat([sum_df,df],axis=0)
    
    return sum_df

In [11]:
def group_sector(df):
    
    df['group']=df['main_secto']

    df.loc[df['group']=='arabs_behined_seperation_wall','group']='Arab'

    df.loc[df['group']=='arabs_behined_seperation_wall','group']='Arab'
    return df

In [12]:
def compare_scenarios(sum_df,col_index):
    sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

    sum_df=sum_df.fillna(0)

    sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

    sum_df=sum_df.loc[ ~(sum_df == 0).all(axis=1),:]
    
    return sum_df

### העלת קבצים

In [13]:
save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\join_forecast'

jtmt=up_load_df(save_file,'230720_forecast_2020_till_2050_jtmt')

In [14]:
jtmt=jtmt.query('zonetype!="Palestinian"')

In [15]:
save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\bau\join_forecast'

bau=up_load_df(save_file,'230720_forecast_2020_till_2050_bau')

In [16]:
bau=bau.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [17]:
save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\iplan\join_forecast'

iplan=up_load_df(save_file,'230720_forecast_2020_till_2050_iplan')

In [18]:
iplan=iplan.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [19]:
save_file=r'W:\Model Versions\3.1\Forecasts'

v3=up_load_df(save_file,'v3_for_comparison_to_v4')

In [20]:
v3=v3.query('zonetype!="Palestinian"')

In [21]:
save_file=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\נתוני עבר'

past_2010=up_load_df(save_file,'נתוני ישובים ומועצות  2010 - עיבוד')

In [22]:
save_file=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\נתוני עבר'

past_2000=up_load_df(save_file,'נתוני ישובים ומועצות  2000 -עיבוד')

In [23]:
save_file=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\נתוני עבר'

past_2020=up_load_df(save_file,'נתוני רשויות 2020 - עיבוד')

In [24]:
gov=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\שיתופי פעולה\תחזיות כלל ארציות מועצה הלאומית לכלכלה\210512_תוצאות_סופיות','טיוטת תוצאות לתרחישי אוכלוסייה אזוריים - 2020-2050 - טיוטה')

## אחוז תרחיש מכלל ארצי

In [25]:
def gov_goal_by_year(sum_df,df,year,scenario_True,machoz):
    
    
    
    goal=df.query('proj=="medium" & housing_adjusted ==@scenario_True & year==@year')['pop'].sum().item()
    
    data = [ {'scenario': 'iplan' if scenario_True else 'bau', 'year': year,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df
    

In [26]:
def gov_goal_by_year_machoz(sum_df,df,year,scenario_True,machoz_name):
    
    
    
    goal=df.query('proj=="medium" & housing_adjusted ==@scenario_True & year==@year & district==@machoz_name')['pop'].sum().item()
    
    data = [ {'scenario': 'iplan' if scenario_True else 'bau', 'year': year,'zonetype':machoz_name,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df
    

In [27]:
def jtmt_forecast_by_year_machoz(sum_df,jtmt,year,machoz_name):
    
    
    
    goal=jtmt.query('zonetype==@machoz_name')['pop_without_dorms_yeshiva_{}'.format(year)].sum().item()
    
    data = [ {'scenario': 'jtmt' , 'year': year,'zonetype':machoz_name,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df
    

In [28]:
sum_df = pd.DataFrame()
for t in [True,False]:
    for m in ["Jerusalem District","Yehuda and Shomron"]:
        for y in list(range(2020,2051,5)):
            sum_df=gov_goal_by_year_machoz(sum_df,gov,y,t,m)



In [29]:
for m in ["Jerusalem","Judea and Samaria"]:
        for y in list(range(2025,2051,5)):
            sum_df=jtmt_forecast_by_year_machoz(sum_df,jtmt,y,m)


In [30]:
sum_df.loc[sum_df['zonetype']=="Jerusalem",'zonetype']="Jerusalem District"

In [31]:
sum_df.loc[sum_df['zonetype']=="Judea and Samaria",'zonetype']="Yehuda and Shomron"

In [32]:
df=sum_df.pivot_table(index=['year'],columns=['zonetype','scenario'],aggfunc=sum)

In [33]:
sum_total = pd.DataFrame()
for t in [True,False]:
    for y in list(range(2020,2051,5)):
        sum_total=gov_goal_by_year(sum_total,gov,y,t)

sum_total=sum_total.pivot_table(index=['year'],columns=['scenario'],aggfunc=sum)

TypeError: gov_goal_by_year() missing 1 required positional argument: 'machoz'

In [ ]:
df['Totatl_pop_il']=sum_total['pop']['bau']

In [ ]:
df.to_excel('scenario_precent_of_pop_il.xlsx')

## נתוני עבר

In [34]:
sum_df = pd.DataFrame()

col_index=['zonetype','muni_name']

col_need=[ 'pop_without_dorms_yeshiva','pop_0_20','pop_21_45','pop_45_65','pop_65+','student','jew_precent']

sum_df=sum_scenarios(sum_df,past_2000,'2000',col_index,col_need,2)

sum_df=sum_scenarios(sum_df,past_2010,'2010',col_index,col_need,2)

sum_df=sum_scenarios(sum_df,past_2020,'2020',col_index,col_need,2)

In [35]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df=sum_df.fillna(0)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

sum_df=sum_df.loc[ ~(sum_df == 0).all(axis=1),:]

In [36]:
sum_df.sort_values(col_index).to_excel('past_data_2000till2020_muni.xlsx')

## השוואות בין התרחישים

In [37]:
jtmt_full=jtmt

In [38]:
jtmt=jtmt.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

### אוכלוסייה  לפי מחוז תרחיש כל עשור

In [49]:
col_index=['zonetype','Muni_Heb']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [50]:
sum_df.to_excel('sum_scenario_pop_by_muni_for_pre.xlsx')

In [47]:
col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [48]:
sum_df.to_excel('sum_scenario_pop_by_macoz_for_pre.xlsx')

In [ ]:
sum_df[['scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values('group').to_excel('sum_scenario_group_pop.xlsx')

### אוכלוסייה ותעסוקה לפי מחוז תרחיש בשנת 2050

In [ ]:
col=['zonetype']

col_need=['pop_without_dorms_yeshiva','total_emp']

kayim=round(pd.pivot_table(jtmt,index='zonetype',aggfunc=sum)[col_need],-4)

kayim['year']=2020

In [ ]:
col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(jtmt,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='jtmt'

In [ ]:
sum_df=df

In [ ]:
col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(iplan,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='iplan'

In [ ]:
sum_df=pd.concat([sum_df,df],axis=0)

In [ ]:
col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(bau,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='bau'

In [ ]:
sum_df=pd.concat([sum_df,df],axis=0)

In [ ]:
sum_df=sum_df.sort_values('zonetype')

In [ ]:
col=['scenario','pop_without_dorms_yeshiva_2050', 'total_emp_2050']

In [ ]:
sum_df[col].to_excel('sum_scenario_machoz_pop_emp.xlsx')

In [ ]:
kayim.to_excel('kayim_sum_machoz_pop_emp.xlsx')

### אוכלוסייה  לפי מגזר חומשים ותרחיש

In [ ]:
iplan=group_sector(iplan)

In [ ]:
bau=group_sector(bau)

In [ ]:
jtmt=group_sector(jtmt)

In [ ]:
col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt','group',col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan','group',col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau','group',col_need,-3)

In [209]:
sum_df[['scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values('group').to_excel('sum_scenario_group_pop.xlsx')

### אוכלוסיה לפי תרחיש ורשות

In [173]:
col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(jtmt,index=['zonetype','Muni_Heb'],aggfunc=sum)[col_need],-2)

df['scenario']='jtmt'

sum_df=df

In [174]:
col_index=['zonetype','Muni_Heb']

In [175]:
sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-2)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-2)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva','total_emp'],-2)

In [176]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

In [177]:
sum_df.to_excel('sum_scenario_Muni_Heb_pop_emp.xlsx')

### אוכלוסייה לפי מחוז מגזר חומשים ותרחיש

In [208]:
sum_df = pd.DataFrame()

col_index=['zonetype','group']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df[['scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values(by=col_index).to_excel('sum_scenario_machoz_group_pop.xlsx')

### אוכלוסייה בשכונות של ירושלים לפי מגזר ותרחיש

In [179]:
sum_df = pd.DataFrame()

col_index=['SCHN_NAME','group']

col_need=[ 'pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

In [181]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df=sum_df.fillna(0)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

sum_df=sum_df.loc[ ~(sum_df == 0).all(axis=1),:]

In [182]:
sum_df.to_excel('sum_scenario_jeru_group_pop.xlsx')

### אוכלוסייה  לפי מגזר ותרחיש השוואה לפי מחוז

In [183]:
sum_df = pd.DataFrame()

col_index=['zonetype','group']

col_need=['pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

compare_scenarios(sum_df,col_index).T.to_excel('sum_scenario_machoz_group_pop_for_precent.xlsx')

### אוכלוסייה  לפי מגזר ותרחיש השוואה לפי העיר ירושלים

In [184]:
sum_df = pd.DataFrame()

col_index=['in_jerusal','group']

col_need=['pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

compare_scenarios(sum_df,col_index).T.to_excel('sum_scenario_jeru_group_pop_for_precent.xlsx')

## השוואות בין הגירסאות

### אוכלוסיה חינוך ותעסוקה סך הכל 

In [185]:
sum_df = pd.DataFrame()

col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040','student','student_2040','uni_students','uni_students_2040','student_yeshiva','student_yeshiva_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3,'v3',col_index,col_need,-3)

In [186]:
sum_df.groupby(by='scenario').sum().T.to_excel('versions_total_pop_emp_student.xlsx')

### אוכלוסיה ותעסוקה לפי מחוז 

In [187]:
sum_df = pd.DataFrame()

col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3,'v3',col_index,col_need,-3)

In [188]:
compare_scenarios(sum_df,col_index).to_excel('versions_machoz_pop_emp.xlsx')

###   חומש אוכלוסייה לפי מגזר 

In [189]:
jtmt_full=group_sector(jtmt_full)

v3=group_sector(v3)

In [207]:
sum_df = pd.DataFrame()

col_index=['group']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040']


sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3,'v3',col_index,col_need,-3)

sum_df.sort_values('group').to_excel('sum_versions_group_pop.xlsx')

###   התפלגות גילים 

In [191]:
v3_age=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\סיכום_תחזיות\תחזיות_V3.3','220303_התפלגות_גילים_V3_3_Published')

v3=v3.merge(v3_age,right_on='taz_V3_2',left_on='Taz_num',how='left')

In [192]:
def sum_age(df,name_col,col_to_sum):
    df[name_col]=df[col].sum(axis=1)
    return df

In [193]:
col=['2020_0-4',
 '2020_5-9',
 '2020_10-14',
 '2020_15-19']

v3=sum_age(v3,'pop_0_20_2020',col)

col=[
 '2020_20-24',
 '2020_25-29',
 '2020_30-34',
 '2020_35-39',
 '2020_40-44']

v3=sum_age(v3,'pop_21_45_2020',col)

col=[
 '2020_45-49',
 '2020_50-54',
 '2020_55-59',
 '2020_60-64']

v3=sum_age(v3,'pop_45_65_2020',col)

col=[
 '2020_65-69',
 '2020_70-74',
 '2020_75+']

v3=sum_age(v3,'pop_65+_2020',col)

In [194]:
col=['2040_0-4',
 '2040_5-9',
 '2040_10-14',
 '2040_15-19']

v3=sum_age(v3,'pop_0_20_2040',col)

col=[
 '2040_20-24',
 '2040_25-29',
 '2040_30-34',
 '2040_35-39',
 '2040_40-44']

v3=sum_age(v3,'pop_21_45_2040',col)

col=[
 '2040_45-49',
 '2040_50-54',
 '2040_55-59',
 '2040_60-64']

v3=sum_age(v3,'pop_45_65_2040',col)

col=[
 '2040_65-69',
 '2040_70-74',
 '2040_75+']

v3=sum_age(v3,'pop_65+_2040',col)

In [195]:
percentage_cols=['pop_0_20_2040','pop_21_45_2040','pop_45_65_2040','pop_65+_2040']

In [196]:
v3[percentage_cols] = v3[percentage_cols].mul(v3['pop_without_dorms_yeshiva_2040'], axis=0)

In [197]:
percentage_cols=['pop_0_20_2020','pop_21_45_2020','pop_45_65_2020','pop_65+_2020']

In [198]:
v3[percentage_cols] = v3[percentage_cols].mul(v3['pop_without_dorms_yeshiva'], axis=0)

In [199]:
jtmt_2040=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\join_forecast','230720_forecast_2040')

jtmt_2040=jtmt_full.merge(jtmt_2040,right_on='TAZ',left_on='Taz_num',how='left')

col=['age0_4',
 'age5_9',
 'age10_14',
 'age15_19',]

jtmt_2040=sum_age(jtmt_2040,'pop_0_20_2040',col)

col=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',]

jtmt_2040=sum_age(jtmt_2040,'pop_21_45_2040',col)

col=[
'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',]

jtmt_2040=sum_age(jtmt_2040,'pop_45_65_2040',col)

col=[
  
 'age65_69',
 'age70_74',
 'age75up',]

jtmt_2040=sum_age(jtmt_2040,'pop_65+_2040',col)

In [200]:
jtmt_2020=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020','forecast_2020_230720')

jtmt_2020=jtmt_full.merge(jtmt_2020,right_on='TAZ',left_on='Taz_num',how='left')

col=['age0_4',
 'age5_9',
 'age10_14',
 'age15_19',]

jtmt_2020=sum_age(jtmt_2020,'pop_0_20_2020',col)

col=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',]

jtmt_2020=sum_age(jtmt_2020,'pop_21_45_2020',col)

col=[
'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',]

jtmt_2020=sum_age(jtmt_2020,'pop_45_65_2020',col)

col=[
  
 'age65_69',
 'age70_74',
 'age75up',]

jtmt_2020=sum_age(jtmt_2020,'pop_65+_2020',col)

In [201]:
jtmt_2020=group_sector(jtmt_2020)

In [202]:
jtmt_2040=group_sector(jtmt_2040)

In [203]:
sum_df = pd.DataFrame()

col_index=['group']

col_need=['pop_0_20_2020',
 'pop_21_45_2020',
 'pop_45_65_2020',
 'pop_65+_2020']

sum_df=sum_scenarios(sum_df,jtmt_2020,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3,'v3',col_index,col_need,-3)

In [204]:
col_need=[
 'pop_0_20_2040',
 'pop_21_45_2040',
 'pop_45_65_2040',
 'pop_65+_2040']

sum_df=sum_scenarios(sum_df,jtmt_2040,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3,'v3',col_index,col_need,-3)

In [205]:
sum_df.groupby(by=['group','scenario']).sum().to_excel('sum_versions_group_age_des.xlsx')